In [2]:
import math
from scipy.constants import Planck, alpha, e, epsilon_0, speed_of_light, m_p, m_n, c
from random import choices
import simpy as simp
import random as rm
import numpy as np

In [3]:
#function for caculating the electric dipole capture
def cap_E(l,R,v):
    M = 1
    h = Planck/(2*math.pi)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Qlmp = ((ke*h)/(M*c))*Qlm
    w = 2*math.pi*v
    E__amp = -((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Qlm+Qlmp))
    Ue = (c/(2*math.pi*(ke**2)))*(abs(E__amp)**2)
    return sig_cap_lm, Ue

#function for calculating the magnetic dipole caputure    
def cap_M(l,R,v):
    M = 1
    h = Planck/(2*math.pi)
    Jl = (3*(R**l))/(l+3)
    ke = ((2*math.pi*v)/c)
    Qlm = ((3*(3.5*(10**-8)))**l)*e
    Mlm = Qlm*(v/c)
    Mlmp = (math.sqrt((10*(h/(M*c*R))**2)*(abs(Qlm)**2)))+Mlm
    w = 2*math.pi*v
    M__amp = +((4*math.pi)/(math.factorial(math.factorial(2*l + 1))))*(((l+1)/l)**0.5)*(ke**(l+2))*Qlm
    sig_cap_lm = ((8*math.pi*(l+1))/(l*math.factorial(math.factorial(2*l + 1))**2))*((ke**(2*l + 1))/h)*(abs(Mlm+Mlmp))
    Um = (c/(2*math.pi*(ke**2)))*(abs(M__amp)**2)
    return sig_cap_lm, Um

In [8]:
counts = 5000
Type = 'hw+D'
def neutron_collisions(env,counts,Type):
    M = 1
    E = 0
    Me = 0
    scat = 0
    testNum = 0
    l = 1
    A = 2
    R = (A**(1/3))*(10**-13)
    h = Planck/(math.pi*2)
    #for the formation of a duetron by neutron, proton capture
    if Type=='n+p':
        for i in range(counts+1):
            print('Test %d' % testNum)
            #neutron energy is randomly chosen from the range 8.5*10^10 and 10^11
            e = rm.randint(8.5*(10**10),1*(10**11))
            yield env.timeout(1)
            h = Planck/(math.pi*2)
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            react = ['cap E','cap M','scat']
            cross = [cap__E/(2*l +1),cap__M/(2*l + 1),1-((cap__E/(2*l +1))-(cap__M/(2*l + 1)))]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e)
            print(win)
            print(cross)
            if win==['cap E']:
                E+=1
                print(UE,UE)
            if win==['scat']:
                scat+=1
            elif win==['cap M']:
                Me+=1
                print(UE,UE)
            print(E,Me,scat,'\n')
            testNum+=1
    #for photodisitegration of the deutron
    elif Type=='hw+D':
        E = 0
        Me = 0
        for i in range(counts+1):
            #photon energy is randomly chosen from this range:
            e = rm.randint(10,50)/(1*(10**44.7))
            k = ((M*e)/(h**2))**(1/2)
            v = (2*h*k)/M
            ke = ((2*math.pi*v)/c)
            cap__E,UE = cap_E(l,R,v)
            cap__M,UM = cap_M(l,R,v)
            dis_E = cap__E*((k/ke)**2)
            dis_M = cap__M*((k/ke)**2)
            react = ['dis E','dis M']
            cross = [dis_E/(2*l +1),dis_M/(2*l + 1)]
            #here the program takes a random weighted choice of the reactions using the probabilities that they will happen
            win=choices(react, cross)
            print(e)
            print(win)
            print(cross)
            if win==['dis E']:
                E+=1
                print(UE,UM)
            elif win==['dis M']:
                Me+=1
                print(UE,UM)
            print(E,Me,'\n')
env = simp.Environment()
env.process(neutron_collisions(env,counts,Type))
env.run(until=counts+2)

3.7909983984408466e-44
['dis E']
[0.1434413552620404, 1.639086358733777e-30]
3.6499658180695065e-164 3.6499658180695065e-164
1 0 

3.9905246299377333e-44
['dis E']
[0.15491339504485205, 1.7806307395057666e-30]
4.044283454924664e-164 4.044283454924664e-164
2 0 

5.187682018919053e-44
['dis E']
[0.2296169798634593, 2.7250895496006422e-30]
6.834839038822681e-164 6.834839038822681e-164
3 0 

4.988155787422167e-44
['dis E']
[0.2164980512159802, 2.5566178129780706e-30]
6.319192898319791e-164 6.319192898319791e-164
4 0 

8.9786804173599e-44
['dis E']
[0.5228327082763755, 6.706394889213086e-30]
2.0474184990556112e-163 2.0474184990556112e-163
5 0 

5.586734481912827e-44
['dis E']
[0.25661440127572854, 3.0749327006782894e-30]
7.92679557165234e-164 7.92679557165234e-164
6 0 

9.178206648856787e-44
['dis E']
[0.540356930450522, 6.955050862496204e-30]
2.1394259476551453e-163 2.1394259476551453e-163
7 0 

2.5938410094595266e-44
['dis E']
[0.08118186176851347, 8.912052607463373e-31]
1.708709759705668

['dis E']
[0.06318776672793665, 6.827873580526833e-31]
1.2233957451147113e-164 1.2233957451147113e-164
201 0 

4.78862955592528e-44
['dis E']
[0.20363894622185877, 2.392504554886795e-30]
5.823768175091516e-164 5.823768175091516e-164
202 0 

8.9786804173599e-44
['dis E']
[0.5228327082763755, 6.706394889213086e-30]
2.0474184990556112e-163 2.0474184990556112e-163
203 0 

9.57725911185056e-44
['dis E']
[0.5759779191486362, 7.463611287050674e-30]
2.329507270036608e-163 2.329507270036608e-163
204 0 

5.187682018919053e-44
['dis E']
[0.2296169798634593, 2.7250895496006422e-30]
6.834839038822681e-164 6.834839038822681e-164
205 0 

7.581996796881693e-44
['dis E']
[0.4057134200335098, 5.072612235876632e-30]
1.4599863272278035e-163 1.4599863272278035e-163
206 0 

9.377732880353673e-44
['dis E']
[0.5580726836595571, 7.207462357936036e-30]
2.2334555379821442e-163 2.2334555379821442e-163
207 0 

5.786260713409714e-44
['dis E']
[0.2704836232033879, 3.256222262797474e-30]
8.503105963979105e-164 8.5031

[0.20363894622185877, 2.392504554886795e-30]
5.823768175091516e-164 5.823768175091516e-164
353 0 

8.579627954366127e-44
['dis E']
[0.48836739554911146, 6.220424895783744e-30]
1.8694700270389243e-163 1.8694700270389243e-163
354 0 

9.178206648856787e-44
['dis E']
[0.540356930450522, 6.955050862496204e-30]
2.1394259476551453e-163 2.1394259476551453e-163
355 0 

2.5938410094595266e-44
['dis E']
[0.08118186176851347, 8.912052607463373e-31]
1.7087097597056682e-164 1.7087097597056682e-164
356 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.3701579649443234e-30]
2.921994796183068e-164 2.921994796183068e-164
357 0 

9.178206648856787e-44
['dis E']
[0.540356930450522, 6.955050862496204e-30]
2.1394259476551453e-163 2.1394259476551453e-163
358 0 

7.581996796881693e-44
['dis E']
[0.4057134200335098, 5.072612235876632e-30]
1.4599863272278035e-163 1.4599863272278035e-163
359 0 

8.180575491372354e-44
['dis E']
[0.45469456546415077, 5.749709710528597e-30]
1.6996101219320907e-163 1.6996

2.2334555379821442e-163 2.2334555379821442e-163
513 0 

8.180575491372354e-44
['dis E']
[0.45469456546415077, 5.749709710528597e-30]
1.6996101219320907e-163 1.6996101219320907e-163
514 0 

5.38720825041594e-44
['dis E']
[0.24299068464083082, 2.8978752001072985e-30]
7.370706596600205e-164 7.370706596600205e-164
515 0 

4.78862955592528e-44
['dis E']
[0.20363894622185877, 2.392504554886795e-30]
5.823768175091516e-164 5.823768175091516e-164
516 0 

3.1924197039501866e-44
['dis E']
[0.1108470022225818, 1.2429245673908575e-30]
2.5883414111517846e-164 2.5883414111517846e-164
517 0 

8.9786804173599e-44
['dis E']
[0.5228327082763755, 6.706394889213086e-30]
2.0474184990556112e-163 2.0474184990556112e-163
518 0 

7.78152302837858e-44
['dis E']
[0.4218333277082964, 5.294418053109188e-30]
1.5378387837351033e-163 1.5378387837351033e-163
519 0 

4.78862955592528e-44
['dis E']
[0.20363894622185877, 2.392504554886795e-30]
5.823768175091516e-164 5.823768175091516e-164
520 0 

1.9952623149688666e-44
['

9.099637773580491e-164 9.099637773580491e-164
665 0 

9.976311574844334e-44
['dis E']
[0.612348960513968, 7.987051905003614e-30]
2.527677159327916e-163 2.527677159327916e-163
666 0 

5.786260713409714e-44
['dis E']
[0.2704836232033879, 3.256222262797474e-30]
8.503105963979105e-164 8.503105963979105e-164
667 0 

4.3895770929315066e-44
['dis E']
[0.1787219933654308, 2.0775458246783597e-30]
4.893582980458843e-164 4.893582980458843e-164
668 0 

9.57725911185056e-44
['dis E']
[0.5759779191486362, 7.463611287050674e-30]
2.329507270036608e-163 2.329507270036608e-163
669 0 

9.776785343347447e-44
['dis E']
[0.5940706525366495, 7.723480129714713e-30]
2.42758114381853e-163 2.42758114381853e-163
670 0 

6.384839407900373e-44
['dis E']
[0.31352266778315147, 3.825115563946192e-30]
1.035336564460713e-163 1.035336564460713e-163
671 0 

4.589103324428394e-44
['dis E']
[0.1910450248913559, 2.2327970989238155e-30]
5.348564869137864e-164 5.348564869137864e-164
672 0 

2.5938410094595266e-44
['dis E']
[0.

...

4203 0 

4.3895770929315066e-44
['dis E']
[0.1787219933654308, 2.0775458246783597e-30]
4.893582980458843e-164 4.893582980458843e-164
4204 0 

4.589103324428394e-44
['dis E']
[0.1910450248913559, 2.2327970989238155e-30]
5.348564869137864e-164 5.348564869137864e-164
4205 0 

2.39431477796264e-44
['dis E']
[0.07199723989357952, 7.843079669600278e-31]
1.45594204377288e-164 1.45594204377288e-164
4206 0 

2.9928934724533e-44
['dis E']
[0.10061920162150213, 1.1206235588767671e-30]
2.274909443395122e-164 2.274909443395122e-164
4207 0 

3.1924197039501866e-44
['dis E']
[0.1108470022225818, 1.2429245673908575e-30]
2.5883414111517846e-164 2.5883414111517846e-164
4208 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.3701579649443234e-30]
2.921994796183068e-164 2.921994796183068e-164
4209 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.3701579649443234e-30]
2.921994796183068e-164 2.921994796183068e-164
4210 0 

4.589103324428394e-44
['dis E']
[0.1910450248913559, 2.23279709

4409 0 

6.783891870894147e-44
['dis E']
[0.3433700043753325, 4.2248945587768594e-30]
1.1687979184732282e-163 1.1687979184732282e-163
4410 0 

8.779154185863013e-44
['dis E']
[0.5055021338234932, 6.461513126336915e-30]
1.957433192183538e-163 1.957433192183538e-163
4411 0 

9.976311574844334e-44
['dis E']
[0.612348960513968, 7.987051905003614e-30]
2.527677159327916e-163 2.527677159327916e-163
4412 0 

3.9905246299377333e-44
['dis E']
[0.15491339504485205, 1.7806307395057666e-30]
4.044283454924664e-164 4.044283454924664e-164
4413 0 

5.586734481912827e-44
['dis E']
[0.25661440127572854, 3.0749327006782894e-30]
7.92679557165234e-164 7.92679557165234e-164
4414 0 

4.78862955592528e-44
['dis E']
[0.20363894622185877, 2.392504554886795e-30]
5.823768175091516e-164 5.823768175091516e-164
4415 0 

4.589103324428394e-44
['dis E']
[0.1910450248913559, 2.2327970989238155e-30]
5.348564869137864e-164 5.348564869137864e-164
4416 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.37015796494

4619 0 

9.57725911185056e-44
['dis E']
[0.5759779191486362, 7.463611287050674e-30]
2.329507270036608e-163 2.329507270036608e-163
4620 0 

4.19005086143462e-44
['dis E']
[0.16667594286128604, 1.9268045115825774e-30]
4.458822509054442e-164 4.458822509054442e-164
4621 0 

5.187682018919053e-44
['dis E']
[0.2296169798634593, 2.7250895496006422e-30]
6.834839038822681e-164 6.834839038822681e-164
4622 0 

7.78152302837858e-44
['dis E']
[0.4218333277082964, 5.294418053109188e-30]
1.5378387837351033e-163 1.5378387837351033e-163
4623 0 

6.58436563939726e-44
['dis E']
[0.32833326716878924, 4.022974528411915e-30]
1.1010561706032389e-163 1.1010561706032389e-163
4624 0 

4.19005086143462e-44
['dis E']
[0.16667594286128604, 1.9268045115825774e-30]
4.458822509054442e-164 4.458822509054442e-164
4625 0 

9.178206648856787e-44
['dis E']
[0.540356930450522, 6.955050862496204e-30]
2.1394259476551453e-163 2.1394259476551453e-163
4626 0 

4.3895770929315066e-44
['dis E']
[0.1787219933654308, 2.077545824678

4840 0 

7.581996796881693e-44
['dis E']
[0.4057134200335098, 5.072612235876632e-30]
1.4599863272278035e-163 1.4599863272278035e-163
4841 0 

2.5938410094595266e-44
['dis E']
[0.08118186176851347, 8.912052607463373e-31]
1.7087097597056682e-164 1.7087097597056682e-164
4842 0 

3.7909983984408466e-44
['dis E']
[0.1434413552620404, 1.639086358733777e-30]
3.6499658180695065e-164 3.6499658180695065e-164
4843 0 

9.776785343347447e-44
['dis E']
[0.5940706525366495, 7.723480129714713e-30]
2.42758114381853e-163 2.42758114381853e-163
4844 0 

5.786260713409714e-44
['dis E']
[0.2704836232033879, 3.256222262797474e-30]
8.503105963979105e-164 8.503105963979105e-164
4845 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.3701579649443234e-30]
2.921994796183068e-164 2.921994796183068e-164
4846 0 

4.3895770929315066e-44
['dis E']
[0.1787219933654308, 2.0775458246783597e-30]
4.893582980458843e-164 4.893582980458843e-164
4847 0 

3.59147216694396e-44
['dis E']
[0.1322673754710171, 1.50223804

4976 0 

3.7909983984408466e-44
['dis E']
[0.1434413552620404, 1.639086358733777e-30]
3.6499658180695065e-164 3.6499658180695065e-164
4977 0 

4.988155787422167e-44
['dis E']
[0.2164980512159802, 2.5566178129780706e-30]
6.319192898319791e-164 6.319192898319791e-164
4978 0 

3.1924197039501866e-44
['dis E']
[0.1108470022225818, 1.2429245673908575e-30]
2.5883414111517846e-164 2.5883414111517846e-164
4979 0 

3.3919459354470733e-44
['dis E']
[0.12139962927492602, 1.3701579649443234e-30]
2.921994796183068e-164 2.921994796183068e-164
4980 0 

5.586734481912827e-44
['dis E']
[0.25661440127572854, 3.0749327006782894e-30]
7.92679557165234e-164 7.92679557165234e-164
4981 0 

4.3895770929315066e-44
['dis E']
[0.1787219933654308, 2.0775458246783597e-30]
4.893582980458843e-164 4.893582980458843e-164
4982 0 

7.18294433388792e-44
['dis E']
[0.3741086325012138, 4.640800746708066e-30]
1.3103478393955908e-163 1.3103478393955908e-163
4983 0 

5.38720825041594e-44
['dis E']
[0.24299068464083082, 2.89787